In [192]:
# -------------------------------- DEFINITIONS --------------------------------------------
import os
from pypianoroll import Multitrack, Track
from music21 import converter, corpus, instrument, midi, note, chord, pitch, stream, roman
import pandas as pd
import numpy as np

def open_midi(midi_path, remove_drums):
    # There is an one-line method to read MIDIs
    # but to remove the drums we need to manipulate some
    # low level MIDI events.
    mf = midi.MidiFile()
    mf.open(midi_path)
    mf.read()
    mf.close()
    if (remove_drums):
        for i in range(len(mf.tracks)):
            mf.tracks[i].events = [ev for ev in mf.tracks[i].events if ev.channel != 10]          

    return midi.translate.midiFileToStream(mf)

def note_count(measure, count_dict):
    bass_note = None
    for chord in measure.recurse().getElementsByClass('Chord'):
        # All notes have the same length of its chord parent.
        note_length = chord.quarterLength
        for note in chord.pitches:          
            # If note is "C5", note.name is "C". We use "C5"
            # style to be able to detect more precise inversions.
            note_name = str(note) 
            if (bass_note is None or bass_note.ps > note.ps):
                bass_note = note
                
            if note_name in count_dict:
                count_dict[note_name] += note_length
            else:
                count_dict[note_name] = note_length
        
    return bass_note

def simplify_roman_name(roman_numeral):
    # Chords can get nasty names as "bII#86#6#5",
    # in this method we try to simplify names, even if it ends in
    # a different chord to reduce the chord vocabulary and display
    # chord function clearer.
    ret = roman_numeral.romanNumeral
    inversion_name = None
    inversion = roman_numeral.inversion()
    
    # Checking valid inversions.
    if ((roman_numeral.isTriad() and inversion < 3) or
            (inversion < 4 and
                 (roman_numeral.seventh is not None or roman_numeral.isSeventh()))):
        inversion_name = roman_numeral.inversionName()
        
    if (inversion_name is not None):
        ret = ret + str(inversion_name)
        
    elif (roman_numeral.isDominantSeventh()): ret = ret + "M7"
    elif (roman_numeral.isDiminishedSeventh()): ret = ret + "o7"
    return ret

def harmonic_reduction(midi_file):
    ret = []
    temp_midi = stream.Score()
    temp_midi_chords = midi_file.chordify()
    temp_midi.insert(0, temp_midi_chords)    
    music_key = temp_midi.analyze('key')
    max_notes_per_chord = 4   
    for m in temp_midi_chords.measures(0, None): # None = get all measures.
        if (type(m) != stream.Measure):
            continue
        
        # Here we count all notes length in each measure,
        # get the most frequent ones and try to create a chord with them.
        count_dict = dict()
        bass_note = note_count(m, count_dict)
        if (len(count_dict) < 1):
            ret.append("-") # Empty measure
            continue
        
        sorted_items = sorted(count_dict.items(), key=lambda x:x[1])
        sorted_notes = [item[0] for item in sorted_items[-max_notes_per_chord:]]
        measure_chord = chord.Chord(sorted_notes)
        
        # Convert the chord to the functional roman representation
        # to make its information independent of the music key.
        roman_numeral = roman.romanNumeralFromChord(measure_chord, music_key)
        ret.append(simplify_roman_name(roman_numeral))
        
    return ret

In [193]:
# DEFINE FIRST THE FUNCTIONS OPEN_MIDI, HARMONIC_REDUCTION

def create_interpolation_dataset(folder, save_path, segment_len=500):
    
    '''This function takes all songs in a folder, generates 5s MIDI files with a stride of 1s along each song,
    generates a harmonic reduction and pianoroll of each track and puts everything in a pandas dataframe,
    which is returned'''
    # INPUT
    # folder: name of the folder that contains all the MIDI songs
    # save_path: name of the folder to store the new MIDI segments
    # segment_len: segment length of the new MIDI segments, 250 by default (5s)
    
    d = {'midi_name': [], 'type': [], 'harmonic_reduction': [], 'pianoroll': []} #create empty dictionary
    
    directory = os.listdir(folder)
    directory = directory[:20]
    for k in range(0,len(directory)): # go through all songs
        str_path = folder+'/'
        song_name = directory[k]

        # get song
        mid_multitrack = Multitrack(filename=folder+'/'+song_name)

        # get merged pianoroll
        merged_pianoroll_total = mid_multitrack.get_merged_pianoroll(mode='sum') # merge all tracks #sum or max

        limit = len(merged_pianoroll_total)
        print('limit: ',limit)
        divisions = int(limit/segment_len)
        for i in range(0,divisions):
            if (segment_len*i) <= (limit-3*segment_len):
        
        # One song (I) contains different number of subsongs (II) according to its length.
        # Each subsong (II) contains 3 segments.
        # Each segment (III) contains 250 events, so each subsong contains 750 events.
        
        # Subsongs are not generated as MIDIs, Segments are.
        # name: Name of original song (I) + Number of subsong (II) + Number of segment (III)

            #segments
                roll_1 = merged_pianoroll_total[segment_len*i:segment_len*(i+1)] #partial segments of pianoroll
                roll_2 = merged_pianoroll_total[segment_len*(i+1):segment_len*(i+2)]
                roll_3 = merged_pianoroll_total[segment_len*(i+2):segment_len*(i+3)]
        
            # create a multitrack which only track is the pianoroll set previously (current_roll)
            #create_multitrack = Multitrack(tracks=[Track(pianoroll=current_roll)]) #mode max or sum
            # create 3 MIDI files (each one corresponds to a segment)
                create_multitrack_1 = Multitrack(tracks=[Track(pianoroll=roll_1)]) #mode max or sum
                create_multitrack_2 = Multitrack(tracks=[Track(pianoroll=roll_2)]) #mode max or sum
                create_multitrack_3 = Multitrack(tracks=[Track(pianoroll=roll_3)]) #mode max or sum
        
                #create_multitrack_1.write(save_path+'/'+song_name+'_'+str(i)+'_1.midi') # convert multtrack to midi
                create_multitrack_1.write(save_path+'/'+song_name+'_'+str(i)+'_1.midi') # convert multtrack to midi
                create_multitrack_2.write(save_path+'/'+song_name+'_'+str(i)+'_2.midi') # convert multtrack to midi
                create_multitrack_3.write(save_path+'/'+song_name+'_'+str(i)+'_3.midi') # convert multtrack to midi

                # STORE SEGMENT 1 IN DICTIONARY
                midi_segment_name = save_path+'/'+song_name+'_'+str(i)+'_1.midi'
                final_name = song_name+'_'+str(i)+'_1.midi'
                midi_file = open_midi(midi_segment_name, True) # ----------------------- OPEN_MIDI()
                harmony_midi = harmonic_reduction(midi_file) # -------------------------------------- HARMONIC_REDUCTION()
                d['midi_name'].append(final_name)
                d['type'].append('init')
                d['harmonic_reduction'].append(harmony_midi)
                d['pianoroll'].append(roll_1)
                
                # STORE SEGMENT 2 IN DICTIONARY
                midi_segment_name = save_path+'/'+song_name+'_'+str(i)+'_2.midi'
                final_name = song_name+'_'+str(i)+'_2.midi'
                midi_file = open_midi(midi_segment_name, True) # ----------------------- OPEN_MIDI()
                harmony_midi = harmonic_reduction(midi_file) # -------------------------------------- HARMONIC_REDUCTION()
                d['midi_name'].append(final_name)
                d['type'].append('interpol')
                d['harmonic_reduction'].append(harmony_midi)
                d['pianoroll'].append(roll_2)
                
                # STORE SEGMENT 3 IN DICTIONARY
                midi_segment_name = save_path+'/'+song_name+'_'+str(i)+'_3.midi'
                final_name = song_name+'_'+str(i)+'_3.midi'
                midi_file = open_midi(midi_segment_name, True) # ----------------------- OPEN_MIDI()
                harmony_midi = harmonic_reduction(midi_file) # -------------------------------------- HARMONIC_REDUCTION()
                d['midi_name'].append(final_name)
                d['type'].append('end')
                d['harmonic_reduction'].append(harmony_midi)
                d['pianoroll'].append(roll_3)
                
                #print('subsong', i, 'done')
                #print('from ', segment_len*(i), 'to ', segment_len*(i+3))
        print(song_name)
        print('song',k,'done')
    return d # dictionary

In [194]:
# Dictionary to concatenate to other dictionaries
test_dictionary = create_interpolation_dataset('lahk_midi_tracks', 'lahk_midi_interpolation_10s')
# Dataframe to concatenate
test_df = pd.DataFrame(data=test_dictionary)

limit:  9336
0bc945bdec9b3e295591dec504ebae13_12413.midi
song 0 done
limit:  13368
6bacae2ca62b42674aff60825a1fc0b7_17283.midi
song 1 done
limit:  5208
5386ffb3b559bebd66657123275688d7_453.midi
song 2 done
limit:  11808
4393b768a7347752216f5f99d42aa0b8_7681.midi
song 3 done
limit:  14016
886d10220f2d116c4ab64fa08c7447d1_7611.midi
song 4 done
limit:  15960
5ed33ceaddeecdcf54b85dcb881f0da8_4924.midi
song 5 done
limit:  5400
b6397995c0999bfc7d592acdcc6785bb_19548.midi
song 6 done
limit:  9792
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi
song 7 done
limit:  6624
8cf2ce253ecc786efe65804f1cc3aa8a_5917.midi
song 8 done
limit:  4896
b70360047b6d793a2eb620d3ea935817_18828.midi
song 9 done
limit:  768
ce8b5eb6a2fc7bcbb6b5e5fb1b544673_16145.midi
song 10 done
limit:  5712
e555e8fdc8e67bc5eff16c09c6a82319_2747.midi
song 11 done
limit:  11352
90ec306bcf3443ea3489c7438831eae1_15051.midi
song 12 done
limit:  6816
81f9a956023052513abb877d6b574575_14474.midi
song 13 done
limit:  7512
83a6d271893b6c93f21d

In [197]:
# SAVE CSV
#test_df.to_pickle('dataset_interpolation_10_pickle.csv')
df = pd.read_pickle('dataset_interpolation_10_pickle.csv')
df.head()

,midi_name,type,harmonic_reduction,pianoroll
0,0bc945bdec9b3e295591dec504ebae13_12413.midi_0_...,init,"[V53, IV53, V53, IV53, V53, v]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,0bc945bdec9b3e295591dec504ebae13_12413.midi_0_...,interpol,"[IV53, V53, I53, I53, IV53, II7]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,0bc945bdec9b3e295591dec504ebae13_12413.midi_0_...,end,"[V7, iii, v42, -VII53, i, IV53]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,0bc945bdec9b3e295591dec504ebae13_12413.midi_1_...,init,"[IV53, V53, I53, I53, IV53, II7]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,0bc945bdec9b3e295591dec504ebae13_12413.midi_1_...,interpol,"[V7, iii, v42, -VII53, i, IV53]","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [200]:
type(df['pianoroll'][0])

numpy.ndarray

In [6]:
import pprint
def vectorize_harmony(model, harmonic_reduction):
    # Gets the model vector values for each chord from the reduction.
    word_vecs = []
    for word in harmonic_reduction:
        if word != '-':
            #print('ok')
            try:
                vec = model[word]
                word_vecs.append(vec)
            except KeyError:
                # Ignore, if the word doesn't exist in the vocabulary
                pass
        #else:
        #    pass
        #    print('pass')
    
    # Assuming that document vector is the mean of all the word vectors.
    return np.mean(word_vecs, axis=0)

def cosine_similarity(vecA, vecB):
    # Find the similarity between two vectors based on the dot product.
    csim = np.dot(vecA, vecB) / (np.linalg.norm(vecA) * np.linalg.norm(vecB))
    if np.isnan(np.sum(csim)):
        return 0
    
    return csim

def calculate_similarity_aux(df, model, source_name, target_names=[], threshold=0):
    source_harmo = df[df["midi_name"] == source_name]["harmonic_reduction"].values[0]
    source_vec = vectorize_harmony(model, source_harmo)    
    results = []
    for name in target_names:
        target_harmo = df[df["midi_name"] == name]["harmonic_reduction"].values[0]
        if (len(target_harmo) == 0):
            continue
            
        target_vec = vectorize_harmony(model, target_harmo)       
        sim_score = cosine_similarity(source_vec, target_vec)
        if sim_score > threshold:
            results.append({
                'score' : sim_score,
                'name' : name
            })
                
    # Sort results by score in desc order
    results.sort(key=lambda k : k['score'] , reverse=True)
    return results

def calculate_similarity(df, model, source_name, target_prefix, threshold=0):
    source_midi_names = df[df["midi_name"] == source_name]["midi_name"].values
    if (len(source_midi_names) == 0):
        print(type(source_midi_names))
        print("Invalid source name")
        return
    
    source_midi_name = source_midi_names[0]
    
    target_midi_names = df[df["midi_name"].str.startswith(target_prefix)]["midi_name"].values  
    print(target_midi_names)
    if (len(target_midi_names) == 0):
        print("Invalid target prefix")
        return
    
    return calculate_similarity_aux(df, model, source_midi_name, target_midi_names, threshold)

In [224]:
# import modules & set up logging
import gensim, logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 
model = gensim.models.Word2Vec(test_df["harmonic_reduction"], min_count=2, window=4)

In [227]:
len(jejeje)

20622

In [228]:
model = gensim.models.Word2Vec(jejeje["harmonic_reduction"], min_count=2, window=4)

In [14]:
def evaluate_sim_and_final_create_dataset(pandas_dataframe, save_songs=True, return_similarity_array=False):
    dataframe = pandas_dataframe
    similarities = [0]*len(dataframe)
    count = 0
    similarities = np.empty([1,1000])
    for index,row in dataframe.iterrows():
        #print(index,row)
        if dataframe['type'][index]=='init':
            print(index)
            harmony_1 = dataframe['harmonic_reduction'][index]
            print(dataframe['midi_name'][index])
            harmony_2 = dataframe['harmonic_reduction'][index+2]
            print(dataframe['midi_name'][index+2])

            vec_1 = vectorize_harmony(model, harmony_1)
            vec_2 = vectorize_harmony(model, harmony_2)

            sim_score = cosine_similarity(vec_1, vec_2)
            #print(sim_score)
            #similarities[count]=sim_score
            count+=1
            if save_songs:
                if sim_score <= 0.8 and sim_score > 0.6:
                    create_multitrack_1 = Multitrack(tracks=[Track(pianoroll=dataframe['pianoroll'][index])])
                    create_multitrack_2 = Multitrack(tracks=[Track(pianoroll=dataframe['pianoroll'][index+1])])
                    create_multitrack_3 = Multitrack(tracks=[Track(pianoroll=dataframe['pianoroll'][index+2])])

                    create_multitrack_1.write('after_sim_0.8/'+dataframe['midi_name'][index])
                    create_multitrack_2.write('after_sim_0.8/'+dataframe['midi_name'][index+1])
                    create_multitrack_3.write('after_sim_0.8/'+dataframe['midi_name'][index+2])
    #if return_similarity_array:
    #return similarities

In [15]:
evaluate_sim_and_final_create_dataset(test_df)

0
0bc945bdec9b3e295591dec504ebae13_12413.midi_0_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_0_3.midi
3
0bc945bdec9b3e295591dec504ebae13_12413.midi_1_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_1_3.midi
6
0bc945bdec9b3e295591dec504ebae13_12413.midi_2_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_2_3.midi
9
0bc945bdec9b3e295591dec504ebae13_12413.midi_3_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_3_3.midi
12
0bc945bdec9b3e295591dec504ebae13_12413.midi_4_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_4_3.midi
15
0bc945bdec9b3e295591dec504ebae13_12413.midi_5_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_5_3.midi
18
0bc945bdec9b3e295591dec504ebae13_12413.midi_6_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_6_3.midi
21
0bc945bdec9b3e295591dec504ebae13_12413.midi_7_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_7_3.midi
24
0bc945bdec9b3e295591dec504ebae13_12413.midi_8_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_8_3.midi
27
0bc945bdec9b3e295591

/home/pablo/miniconda3/envs/ai37/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


75
6bacae2ca62b42674aff60825a1fc0b7_17283.midi_9_1.midi
6bacae2ca62b42674aff60825a1fc0b7_17283.midi_9_3.midi
78
6bacae2ca62b42674aff60825a1fc0b7_17283.midi_10_1.midi
6bacae2ca62b42674aff60825a1fc0b7_17283.midi_10_3.midi
81
6bacae2ca62b42674aff60825a1fc0b7_17283.midi_11_1.midi
6bacae2ca62b42674aff60825a1fc0b7_17283.midi_11_3.midi
84
6bacae2ca62b42674aff60825a1fc0b7_17283.midi_12_1.midi
6bacae2ca62b42674aff60825a1fc0b7_17283.midi_12_3.midi
87
6bacae2ca62b42674aff60825a1fc0b7_17283.midi_13_1.midi
6bacae2ca62b42674aff60825a1fc0b7_17283.midi_13_3.midi
90
6bacae2ca62b42674aff60825a1fc0b7_17283.midi_14_1.midi
6bacae2ca62b42674aff60825a1fc0b7_17283.midi_14_3.midi
93
6bacae2ca62b42674aff60825a1fc0b7_17283.midi_15_1.midi
6bacae2ca62b42674aff60825a1fc0b7_17283.midi_15_3.midi
96
6bacae2ca62b42674aff60825a1fc0b7_17283.midi_16_1.midi
6bacae2ca62b42674aff60825a1fc0b7_17283.midi_16_3.midi
99
6bacae2ca62b42674aff60825a1fc0b7_17283.midi_17_1.midi
6bacae2ca62b42674aff60825a1fc0b7_17283.midi_17_3.midi
102

459
8cf2ce253ecc786efe65804f1cc3aa8a_5917.midi_4_1.midi
8cf2ce253ecc786efe65804f1cc3aa8a_5917.midi_4_3.midi
462
8cf2ce253ecc786efe65804f1cc3aa8a_5917.midi_5_1.midi
8cf2ce253ecc786efe65804f1cc3aa8a_5917.midi_5_3.midi
465
8cf2ce253ecc786efe65804f1cc3aa8a_5917.midi_6_1.midi
8cf2ce253ecc786efe65804f1cc3aa8a_5917.midi_6_3.midi
468
8cf2ce253ecc786efe65804f1cc3aa8a_5917.midi_7_1.midi
8cf2ce253ecc786efe65804f1cc3aa8a_5917.midi_7_3.midi
471
8cf2ce253ecc786efe65804f1cc3aa8a_5917.midi_8_1.midi
8cf2ce253ecc786efe65804f1cc3aa8a_5917.midi_8_3.midi
474
8cf2ce253ecc786efe65804f1cc3aa8a_5917.midi_9_1.midi
8cf2ce253ecc786efe65804f1cc3aa8a_5917.midi_9_3.midi
477
8cf2ce253ecc786efe65804f1cc3aa8a_5917.midi_10_1.midi
8cf2ce253ecc786efe65804f1cc3aa8a_5917.midi_10_3.midi
480
b70360047b6d793a2eb620d3ea935817_18828.midi_0_1.midi
b70360047b6d793a2eb620d3ea935817_18828.midi_0_3.midi
483
b70360047b6d793a2eb620d3ea935817_18828.midi_1_1.midi
b70360047b6d793a2eb620d3ea935817_18828.midi_1_3.midi
486
b70360047b6d793a2e

762
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_20_1.midi
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_20_3.midi
765
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_21_1.midi
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_21_3.midi
768
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_22_1.midi
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_22_3.midi
771
58c48a5ba8264298f7df28ebf90c5476_14958.midi_0_1.midi
58c48a5ba8264298f7df28ebf90c5476_14958.midi_0_3.midi
774
58c48a5ba8264298f7df28ebf90c5476_14958.midi_1_1.midi
58c48a5ba8264298f7df28ebf90c5476_14958.midi_1_3.midi
777
58c48a5ba8264298f7df28ebf90c5476_14958.midi_2_1.midi
58c48a5ba8264298f7df28ebf90c5476_14958.midi_2_3.midi
780
beea331af3631542778128dca2139be3_8281.midi_0_1.midi
beea331af3631542778128dca2139be3_8281.midi_0_3.midi
783
beea331af3631542778128dca2139be3_8281.midi_1_1.midi
beea331af3631542778128dca2139be3_8281.midi_1_3.midi
786
beea331af3631542778128dca2139be3_8281.midi_2_1.midi
beea331af3631542778128dca2139be3_8281.midi_2_3.midi
789
beea331af363

...

19197
0c2600766d5c277b6e66692e4007d0d5_1133.midi_6_1.midi
0c2600766d5c277b6e66692e4007d0d5_1133.midi_6_3.midi
19200
0c2600766d5c277b6e66692e4007d0d5_1133.midi_7_1.midi
0c2600766d5c277b6e66692e4007d0d5_1133.midi_7_3.midi
19203
0c2600766d5c277b6e66692e4007d0d5_1133.midi_8_1.midi
0c2600766d5c277b6e66692e4007d0d5_1133.midi_8_3.midi
19206
0c2600766d5c277b6e66692e4007d0d5_1133.midi_9_1.midi
0c2600766d5c277b6e66692e4007d0d5_1133.midi_9_3.midi
19209
0c2600766d5c277b6e66692e4007d0d5_1133.midi_10_1.midi
0c2600766d5c277b6e66692e4007d0d5_1133.midi_10_3.midi
19212
0c2600766d5c277b6e66692e4007d0d5_1133.midi_11_1.midi
0c2600766d5c277b6e66692e4007d0d5_1133.midi_11_3.midi
19215
0c2600766d5c277b6e66692e4007d0d5_1133.midi_12_1.midi
0c2600766d5c277b6e66692e4007d0d5_1133.midi_12_3.midi
19218
0c2600766d5c277b6e66692e4007d0d5_1133.midi_13_1.midi
0c2600766d5c277b6e66692e4007d0d5_1133.midi_13_3.midi
19221
0c2600766d5c277b6e66692e4007d0d5_1133.midi_14_1.midi
0c2600766d5c277b6e66692e4007d0d5_1133.midi_14_3.midi


19521
f3afc03060463c0d91118e99a858dd94_13570.midi_14_1.midi
f3afc03060463c0d91118e99a858dd94_13570.midi_14_3.midi
19524
f3afc03060463c0d91118e99a858dd94_13570.midi_15_1.midi
f3afc03060463c0d91118e99a858dd94_13570.midi_15_3.midi
19527
29f1e597771e9acdc1661ec1a191594c_2838.midi_0_1.midi
29f1e597771e9acdc1661ec1a191594c_2838.midi_0_3.midi
19530
29f1e597771e9acdc1661ec1a191594c_2838.midi_1_1.midi
29f1e597771e9acdc1661ec1a191594c_2838.midi_1_3.midi
19533
29f1e597771e9acdc1661ec1a191594c_2838.midi_2_1.midi
29f1e597771e9acdc1661ec1a191594c_2838.midi_2_3.midi
19536
29f1e597771e9acdc1661ec1a191594c_2838.midi_3_1.midi
29f1e597771e9acdc1661ec1a191594c_2838.midi_3_3.midi
19539
29f1e597771e9acdc1661ec1a191594c_2838.midi_4_1.midi
29f1e597771e9acdc1661ec1a191594c_2838.midi_4_3.midi
19542
29f1e597771e9acdc1661ec1a191594c_2838.midi_5_1.midi
29f1e597771e9acdc1661ec1a191594c_2838.midi_5_3.midi
19545
29f1e597771e9acdc1661ec1a191594c_2838.midi_6_1.midi
29f1e597771e9acdc1661ec1a191594c_2838.midi_6_3.midi
19

19773
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_1_1.midi
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_1_3.midi
19776
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_2_1.midi
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_2_3.midi
19779
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_3_1.midi
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_3_3.midi
19782
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_4_1.midi
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_4_3.midi
19785
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_5_1.midi
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_5_3.midi
19788
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_6_1.midi
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_6_3.midi
19791
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_7_1.midi
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_7_3.midi
19794
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_8_1.midi
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_8_3.midi
19797
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_9_1.midi
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_9_3.midi
19800
5367

20037
b23b9e621f78077e5baa5563cdfc94d0_12739.midi_0_1.midi
b23b9e621f78077e5baa5563cdfc94d0_12739.midi_0_3.midi
20040
b23b9e621f78077e5baa5563cdfc94d0_12739.midi_1_1.midi
b23b9e621f78077e5baa5563cdfc94d0_12739.midi_1_3.midi
20043
b23b9e621f78077e5baa5563cdfc94d0_12739.midi_2_1.midi
b23b9e621f78077e5baa5563cdfc94d0_12739.midi_2_3.midi
20046
b23b9e621f78077e5baa5563cdfc94d0_12739.midi_3_1.midi
b23b9e621f78077e5baa5563cdfc94d0_12739.midi_3_3.midi
20049
b23b9e621f78077e5baa5563cdfc94d0_12739.midi_4_1.midi
b23b9e621f78077e5baa5563cdfc94d0_12739.midi_4_3.midi
20052
b23b9e621f78077e5baa5563cdfc94d0_12739.midi_5_1.midi
b23b9e621f78077e5baa5563cdfc94d0_12739.midi_5_3.midi
20055
b23b9e621f78077e5baa5563cdfc94d0_12739.midi_6_1.midi
b23b9e621f78077e5baa5563cdfc94d0_12739.midi_6_3.midi
20058
b23b9e621f78077e5baa5563cdfc94d0_12739.midi_7_1.midi
b23b9e621f78077e5baa5563cdfc94d0_12739.midi_7_3.midi
20061
b23b9e621f78077e5baa5563cdfc94d0_12739.midi_8_1.midi
b23b9e621f78077e5baa5563cdfc94d0_12739.midi_8

20271
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_16_1.midi
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_16_3.midi
20274
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_17_1.midi
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_17_3.midi
20277
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_18_1.midi
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_18_3.midi
20280
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_19_1.midi
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_19_3.midi
20283
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_20_1.midi
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_20_3.midi
20286
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_21_1.midi
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_21_3.midi
20289
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_22_1.midi
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_22_3.midi
20292
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_23_1.midi
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_23_3.midi
20295
943e84f1c27ba700b728e39c4e08cbfb_17712.midi_0_1.midi
943e84f1c27ba700b728e39c4e08c

In [12]:
def evaluate_sim_and_final_create_dataset(pandas_dataframe, save_songs=True, return_similarity_array=False):
    dataframe = pandas_dataframe
    similarities = [0]*len(dataframe)
    count = 0
    similarities = np.empty([1,1000])
    for index,row in dataframe.iterrows():
        #print(index,row)
        if dataframe['type'][index]=='init':
            print(index)
            harmony_1 = dataframe['harmonic_reduction'][index]
            print(dataframe['midi_name'][index])
            harmony_2 = dataframe['harmonic_reduction'][index+2]
            print(dataframe['midi_name'][index+2])

            vec_1 = vectorize_harmony(model, harmony_1)
            vec_2 = vectorize_harmony(model, harmony_2)

            sim_score = cosine_similarity(vec_1, vec_2)
            #print(sim_score)
            #similarities[count]=sim_score
            count+=1
            if save_songs:
                if sim_score <= 0.6 and sim_score > 0.4:
                    create_multitrack_1 = Multitrack(tracks=[Track(pianoroll=dataframe['pianoroll'][index])])
                    create_multitrack_2 = Multitrack(tracks=[Track(pianoroll=dataframe['pianoroll'][index+1])])
                    create_multitrack_3 = Multitrack(tracks=[Track(pianoroll=dataframe['pianoroll'][index+2])])

                    create_multitrack_1.write('after_sim_0.6/'+dataframe['midi_name'][index])
                    create_multitrack_2.write('after_sim_0.6/'+dataframe['midi_name'][index+1])
                    create_multitrack_3.write('after_sim_0.6/'+dataframe['midi_name'][index+2])
    #if return_similarity_array:
    #return similarities

In [13]:
evaluate_sim_and_final_create_dataset(test_df)

0
0bc945bdec9b3e295591dec504ebae13_12413.midi_0_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_0_3.midi
3
0bc945bdec9b3e295591dec504ebae13_12413.midi_1_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_1_3.midi
6
0bc945bdec9b3e295591dec504ebae13_12413.midi_2_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_2_3.midi
9
0bc945bdec9b3e295591dec504ebae13_12413.midi_3_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_3_3.midi
12
0bc945bdec9b3e295591dec504ebae13_12413.midi_4_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_4_3.midi
15
0bc945bdec9b3e295591dec504ebae13_12413.midi_5_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_5_3.midi
18
0bc945bdec9b3e295591dec504ebae13_12413.midi_6_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_6_3.midi
21
0bc945bdec9b3e295591dec504ebae13_12413.midi_7_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_7_3.midi
24
0bc945bdec9b3e295591dec504ebae13_12413.midi_8_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_8_3.midi
27
0bc945bdec9b3e295591

/home/pablo/miniconda3/envs/ai37/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


108
6bacae2ca62b42674aff60825a1fc0b7_17283.midi_20_1.midi
6bacae2ca62b42674aff60825a1fc0b7_17283.midi_20_3.midi
111
6bacae2ca62b42674aff60825a1fc0b7_17283.midi_21_1.midi
6bacae2ca62b42674aff60825a1fc0b7_17283.midi_21_3.midi
114
6bacae2ca62b42674aff60825a1fc0b7_17283.midi_22_1.midi
6bacae2ca62b42674aff60825a1fc0b7_17283.midi_22_3.midi
117
6bacae2ca62b42674aff60825a1fc0b7_17283.midi_23_1.midi
6bacae2ca62b42674aff60825a1fc0b7_17283.midi_23_3.midi
120
5386ffb3b559bebd66657123275688d7_453.midi_0_1.midi
5386ffb3b559bebd66657123275688d7_453.midi_0_3.midi
123
5386ffb3b559bebd66657123275688d7_453.midi_1_1.midi
5386ffb3b559bebd66657123275688d7_453.midi_1_3.midi
126
5386ffb3b559bebd66657123275688d7_453.midi_2_1.midi
5386ffb3b559bebd66657123275688d7_453.midi_2_3.midi
129
5386ffb3b559bebd66657123275688d7_453.midi_3_1.midi
5386ffb3b559bebd66657123275688d7_453.midi_3_3.midi
132
5386ffb3b559bebd66657123275688d7_453.midi_4_1.midi
5386ffb3b559bebd66657123275688d7_453.midi_4_3.midi
135
5386ffb3b559bebd66

405
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_3_1.midi
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_3_3.midi
408
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_4_1.midi
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_4_3.midi
411
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_5_1.midi
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_5_3.midi
414
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_6_1.midi
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_6_3.midi
417
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_7_1.midi
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_7_3.midi
420
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_8_1.midi
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_8_3.midi
423
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_9_1.midi
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_9_3.midi
426
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_10_1.midi
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_10_3.midi
429
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_11_1.midi
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_11_3.midi
432
89

714
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_4_1.midi
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_4_3.midi
717
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_5_1.midi
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_5_3.midi
720
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_6_1.midi
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_6_3.midi
723
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_7_1.midi
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_7_3.midi
726
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_8_1.midi
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_8_3.midi
729
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_9_1.midi
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_9_3.midi
732
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_10_1.midi
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_10_3.midi
735
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_11_1.midi
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_11_3.midi
738
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_12_1.midi
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_12_3.midi
741
97d3bb31e9f294cec5

...

19167
16a9b1760c034f6100e39e4f97a6c6fa_9513.midi_18_1.midi
16a9b1760c034f6100e39e4f97a6c6fa_9513.midi_18_3.midi
19170
16a9b1760c034f6100e39e4f97a6c6fa_9513.midi_19_1.midi
16a9b1760c034f6100e39e4f97a6c6fa_9513.midi_19_3.midi
19173
16a9b1760c034f6100e39e4f97a6c6fa_9513.midi_20_1.midi
16a9b1760c034f6100e39e4f97a6c6fa_9513.midi_20_3.midi
19176
16a9b1760c034f6100e39e4f97a6c6fa_9513.midi_21_1.midi
16a9b1760c034f6100e39e4f97a6c6fa_9513.midi_21_3.midi
19179
0c2600766d5c277b6e66692e4007d0d5_1133.midi_0_1.midi
0c2600766d5c277b6e66692e4007d0d5_1133.midi_0_3.midi
19182
0c2600766d5c277b6e66692e4007d0d5_1133.midi_1_1.midi
0c2600766d5c277b6e66692e4007d0d5_1133.midi_1_3.midi
19185
0c2600766d5c277b6e66692e4007d0d5_1133.midi_2_1.midi
0c2600766d5c277b6e66692e4007d0d5_1133.midi_2_3.midi
19188
0c2600766d5c277b6e66692e4007d0d5_1133.midi_3_1.midi
0c2600766d5c277b6e66692e4007d0d5_1133.midi_3_3.midi
19191
0c2600766d5c277b6e66692e4007d0d5_1133.midi_4_1.midi
0c2600766d5c277b6e66692e4007d0d5_1133.midi_4_3.midi
19

19536
29f1e597771e9acdc1661ec1a191594c_2838.midi_3_1.midi
29f1e597771e9acdc1661ec1a191594c_2838.midi_3_3.midi
19539
29f1e597771e9acdc1661ec1a191594c_2838.midi_4_1.midi
29f1e597771e9acdc1661ec1a191594c_2838.midi_4_3.midi
19542
29f1e597771e9acdc1661ec1a191594c_2838.midi_5_1.midi
29f1e597771e9acdc1661ec1a191594c_2838.midi_5_3.midi
19545
29f1e597771e9acdc1661ec1a191594c_2838.midi_6_1.midi
29f1e597771e9acdc1661ec1a191594c_2838.midi_6_3.midi
19548
29f1e597771e9acdc1661ec1a191594c_2838.midi_7_1.midi
29f1e597771e9acdc1661ec1a191594c_2838.midi_7_3.midi
19551
29f1e597771e9acdc1661ec1a191594c_2838.midi_8_1.midi
29f1e597771e9acdc1661ec1a191594c_2838.midi_8_3.midi
19554
29f1e597771e9acdc1661ec1a191594c_2838.midi_9_1.midi
29f1e597771e9acdc1661ec1a191594c_2838.midi_9_3.midi
19557
29f1e597771e9acdc1661ec1a191594c_2838.midi_10_1.midi
29f1e597771e9acdc1661ec1a191594c_2838.midi_10_3.midi
19560
29f1e597771e9acdc1661ec1a191594c_2838.midi_11_1.midi
29f1e597771e9acdc1661ec1a191594c_2838.midi_11_3.midi
19563


19908
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_3_1.midi
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_3_3.midi
19911
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_4_1.midi
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_4_3.midi
19914
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_5_1.midi
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_5_3.midi
19917
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_6_1.midi
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_6_3.midi
19920
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_7_1.midi
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_7_3.midi
19923
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_8_1.midi
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_8_3.midi
19926
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_9_1.midi
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_9_3.midi
19929
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_10_1.midi
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_10_3.midi
19932
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_11_1.midi
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_11_3.midi
19935


20220
0bb8f4fc16294bf3094669844e33569a_1073.midi_18_1.midi
0bb8f4fc16294bf3094669844e33569a_1073.midi_18_3.midi
20223
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_0_1.midi
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_0_3.midi
20226
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_1_1.midi
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_1_3.midi
20229
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_2_1.midi
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_2_3.midi
20232
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_3_1.midi
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_3_3.midi
20235
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_4_1.midi
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_4_3.midi
20238
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_5_1.midi
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_5_3.midi
20241
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_6_1.midi
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_6_3.midi
20244
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_7_1.midi
6bacae2ca62b42674aff60825a1fc0b7_12858.midi_7

20526
4e19e3897f314d72146dbb2e5207d956_11585.midi_5_1.midi
4e19e3897f314d72146dbb2e5207d956_11585.midi_5_3.midi
20529
4e19e3897f314d72146dbb2e5207d956_11585.midi_6_1.midi
4e19e3897f314d72146dbb2e5207d956_11585.midi_6_3.midi
20532
4e19e3897f314d72146dbb2e5207d956_11585.midi_7_1.midi
4e19e3897f314d72146dbb2e5207d956_11585.midi_7_3.midi
20535
4e19e3897f314d72146dbb2e5207d956_11585.midi_8_1.midi
4e19e3897f314d72146dbb2e5207d956_11585.midi_8_3.midi
20538
4e19e3897f314d72146dbb2e5207d956_11585.midi_9_1.midi
4e19e3897f314d72146dbb2e5207d956_11585.midi_9_3.midi
20541
4e19e3897f314d72146dbb2e5207d956_11585.midi_10_1.midi
4e19e3897f314d72146dbb2e5207d956_11585.midi_10_3.midi
20544
4e19e3897f314d72146dbb2e5207d956_11585.midi_11_1.midi
4e19e3897f314d72146dbb2e5207d956_11585.midi_11_3.midi
20547
4e19e3897f314d72146dbb2e5207d956_11585.midi_12_1.midi
4e19e3897f314d72146dbb2e5207d956_11585.midi_12_3.midi
20550
4e19e3897f314d72146dbb2e5207d956_11585.midi_13_1.midi
4e19e3897f314d72146dbb2e5207d956_11585

In [10]:
def evaluate_sim_and_final_create_dataset(pandas_dataframe, save_songs=True, return_similarity_array=False):
    dataframe = pandas_dataframe
    similarities = [0]*len(dataframe)
    count = 0
    similarities = np.empty([1,1000])
    for index,row in dataframe.iterrows():
        #print(index,row)
        if dataframe['type'][index]=='init':
            print(index)
            harmony_1 = dataframe['harmonic_reduction'][index]
            print(dataframe['midi_name'][index])
            harmony_2 = dataframe['harmonic_reduction'][index+2]
            print(dataframe['midi_name'][index+2])

            vec_1 = vectorize_harmony(model, harmony_1)
            vec_2 = vectorize_harmony(model, harmony_2)

            sim_score = cosine_similarity(vec_1, vec_2)
            #print(sim_score)
            #similarities[count]=sim_score
            count+=1
            if save_songs:
                if sim_score <= 0.4 and sim_score > 0.2:
                    create_multitrack_1 = Multitrack(tracks=[Track(pianoroll=dataframe['pianoroll'][index])])
                    create_multitrack_2 = Multitrack(tracks=[Track(pianoroll=dataframe['pianoroll'][index+1])])
                    create_multitrack_3 = Multitrack(tracks=[Track(pianoroll=dataframe['pianoroll'][index+2])])

                    create_multitrack_1.write('after_sim_0.4/'+dataframe['midi_name'][index])
                    create_multitrack_2.write('after_sim_0.4/'+dataframe['midi_name'][index+1])
                    create_multitrack_3.write('after_sim_0.4/'+dataframe['midi_name'][index+2])
    #if return_similarity_array:
    #return similarities

In [11]:
evaluate_sim_and_final_create_dataset(test_df)

0
0bc945bdec9b3e295591dec504ebae13_12413.midi_0_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_0_3.midi
3
0bc945bdec9b3e295591dec504ebae13_12413.midi_1_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_1_3.midi
6
0bc945bdec9b3e295591dec504ebae13_12413.midi_2_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_2_3.midi
9
0bc945bdec9b3e295591dec504ebae13_12413.midi_3_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_3_3.midi
12
0bc945bdec9b3e295591dec504ebae13_12413.midi_4_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_4_3.midi
15
0bc945bdec9b3e295591dec504ebae13_12413.midi_5_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_5_3.midi
18
0bc945bdec9b3e295591dec504ebae13_12413.midi_6_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_6_3.midi
21
0bc945bdec9b3e295591dec504ebae13_12413.midi_7_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_7_3.midi
24
0bc945bdec9b3e295591dec504ebae13_12413.midi_8_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_8_3.midi
27
0bc945bdec9b3e295591

/home/pablo/miniconda3/envs/ai37/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


213
886d10220f2d116c4ab64fa08c7447d1_7611.midi_2_1.midi
886d10220f2d116c4ab64fa08c7447d1_7611.midi_2_3.midi
216
886d10220f2d116c4ab64fa08c7447d1_7611.midi_3_1.midi
886d10220f2d116c4ab64fa08c7447d1_7611.midi_3_3.midi
219
886d10220f2d116c4ab64fa08c7447d1_7611.midi_4_1.midi
886d10220f2d116c4ab64fa08c7447d1_7611.midi_4_3.midi
222
886d10220f2d116c4ab64fa08c7447d1_7611.midi_5_1.midi
886d10220f2d116c4ab64fa08c7447d1_7611.midi_5_3.midi
225
886d10220f2d116c4ab64fa08c7447d1_7611.midi_6_1.midi
886d10220f2d116c4ab64fa08c7447d1_7611.midi_6_3.midi
228
886d10220f2d116c4ab64fa08c7447d1_7611.midi_7_1.midi
886d10220f2d116c4ab64fa08c7447d1_7611.midi_7_3.midi
231
886d10220f2d116c4ab64fa08c7447d1_7611.midi_8_1.midi
886d10220f2d116c4ab64fa08c7447d1_7611.midi_8_3.midi
234
886d10220f2d116c4ab64fa08c7447d1_7611.midi_9_1.midi
886d10220f2d116c4ab64fa08c7447d1_7611.midi_9_3.midi
237
886d10220f2d116c4ab64fa08c7447d1_7611.midi_10_1.midi
886d10220f2d116c4ab64fa08c7447d1_7611.midi_10_3.midi
240
886d10220f2d116c4ab64f

465
8cf2ce253ecc786efe65804f1cc3aa8a_5917.midi_6_1.midi
8cf2ce253ecc786efe65804f1cc3aa8a_5917.midi_6_3.midi
468
8cf2ce253ecc786efe65804f1cc3aa8a_5917.midi_7_1.midi
8cf2ce253ecc786efe65804f1cc3aa8a_5917.midi_7_3.midi
471
8cf2ce253ecc786efe65804f1cc3aa8a_5917.midi_8_1.midi
8cf2ce253ecc786efe65804f1cc3aa8a_5917.midi_8_3.midi
474
8cf2ce253ecc786efe65804f1cc3aa8a_5917.midi_9_1.midi
8cf2ce253ecc786efe65804f1cc3aa8a_5917.midi_9_3.midi
477
8cf2ce253ecc786efe65804f1cc3aa8a_5917.midi_10_1.midi
8cf2ce253ecc786efe65804f1cc3aa8a_5917.midi_10_3.midi
480
b70360047b6d793a2eb620d3ea935817_18828.midi_0_1.midi
b70360047b6d793a2eb620d3ea935817_18828.midi_0_3.midi
483
b70360047b6d793a2eb620d3ea935817_18828.midi_1_1.midi
b70360047b6d793a2eb620d3ea935817_18828.midi_1_3.midi
486
b70360047b6d793a2eb620d3ea935817_18828.midi_2_1.midi
b70360047b6d793a2eb620d3ea935817_18828.midi_2_3.midi
489
b70360047b6d793a2eb620d3ea935817_18828.midi_3_1.midi
b70360047b6d793a2eb620d3ea935817_18828.midi_3_3.midi
492
b70360047b6d79

858
584017a4bad5e83c04686c0f2798fc31_17114.midi_10_1.midi
584017a4bad5e83c04686c0f2798fc31_17114.midi_10_3.midi
861
584017a4bad5e83c04686c0f2798fc31_17114.midi_11_1.midi
584017a4bad5e83c04686c0f2798fc31_17114.midi_11_3.midi
864
584017a4bad5e83c04686c0f2798fc31_17114.midi_12_1.midi
584017a4bad5e83c04686c0f2798fc31_17114.midi_12_3.midi
867
a4dad8ac2175fae8bafa64242069b904_3979.midi_0_1.midi
a4dad8ac2175fae8bafa64242069b904_3979.midi_0_3.midi
870
a4dad8ac2175fae8bafa64242069b904_3979.midi_1_1.midi
a4dad8ac2175fae8bafa64242069b904_3979.midi_1_3.midi
873
a4dad8ac2175fae8bafa64242069b904_3979.midi_2_1.midi
a4dad8ac2175fae8bafa64242069b904_3979.midi_2_3.midi
876
a4dad8ac2175fae8bafa64242069b904_3979.midi_3_1.midi
a4dad8ac2175fae8bafa64242069b904_3979.midi_3_3.midi
879
a4dad8ac2175fae8bafa64242069b904_3979.midi_4_1.midi
a4dad8ac2175fae8bafa64242069b904_3979.midi_4_3.midi
882
a4dad8ac2175fae8bafa64242069b904_3979.midi_5_1.midi
a4dad8ac2175fae8bafa64242069b904_3979.midi_5_3.midi
885
a4dad8ac2175

...

18987
8bd6c8c8fc62e670d1214d33329e0fd2_10774.midi_9_1.midi
8bd6c8c8fc62e670d1214d33329e0fd2_10774.midi_9_3.midi
18990
8bd6c8c8fc62e670d1214d33329e0fd2_10774.midi_10_1.midi
8bd6c8c8fc62e670d1214d33329e0fd2_10774.midi_10_3.midi
18993
8bd6c8c8fc62e670d1214d33329e0fd2_10774.midi_11_1.midi
8bd6c8c8fc62e670d1214d33329e0fd2_10774.midi_11_3.midi
18996
8bd6c8c8fc62e670d1214d33329e0fd2_10774.midi_12_1.midi
8bd6c8c8fc62e670d1214d33329e0fd2_10774.midi_12_3.midi
18999
8bd6c8c8fc62e670d1214d33329e0fd2_10774.midi_13_1.midi
8bd6c8c8fc62e670d1214d33329e0fd2_10774.midi_13_3.midi
19002
8bd6c8c8fc62e670d1214d33329e0fd2_10774.midi_14_1.midi
8bd6c8c8fc62e670d1214d33329e0fd2_10774.midi_14_3.midi
19005
8bd6c8c8fc62e670d1214d33329e0fd2_10774.midi_15_1.midi
8bd6c8c8fc62e670d1214d33329e0fd2_10774.midi_15_3.midi
19008
8bd6c8c8fc62e670d1214d33329e0fd2_10774.midi_16_1.midi
8bd6c8c8fc62e670d1214d33329e0fd2_10774.midi_16_3.midi
19011
8bd6c8c8fc62e670d1214d33329e0fd2_10774.midi_17_1.midi
8bd6c8c8fc62e670d1214d33329e0f

19245
0c2600766d5c277b6e66692e4007d0d5_1133.midi_22_1.midi
0c2600766d5c277b6e66692e4007d0d5_1133.midi_22_3.midi
19248
0c2600766d5c277b6e66692e4007d0d5_1133.midi_23_1.midi
0c2600766d5c277b6e66692e4007d0d5_1133.midi_23_3.midi
19251
0c2600766d5c277b6e66692e4007d0d5_1133.midi_24_1.midi
0c2600766d5c277b6e66692e4007d0d5_1133.midi_24_3.midi
19254
0c2600766d5c277b6e66692e4007d0d5_1133.midi_25_1.midi
0c2600766d5c277b6e66692e4007d0d5_1133.midi_25_3.midi
19257
0c2600766d5c277b6e66692e4007d0d5_1133.midi_26_1.midi
0c2600766d5c277b6e66692e4007d0d5_1133.midi_26_3.midi
19260
0c2600766d5c277b6e66692e4007d0d5_1133.midi_27_1.midi
0c2600766d5c277b6e66692e4007d0d5_1133.midi_27_3.midi
19263
0c2600766d5c277b6e66692e4007d0d5_1133.midi_28_1.midi
0c2600766d5c277b6e66692e4007d0d5_1133.midi_28_3.midi
19266
0c2600766d5c277b6e66692e4007d0d5_1133.midi_29_1.midi
0c2600766d5c277b6e66692e4007d0d5_1133.midi_29_3.midi
19269
0c2600766d5c277b6e66692e4007d0d5_1133.midi_30_1.midi
0c2600766d5c277b6e66692e4007d0d5_1133.midi_30

19785
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_5_1.midi
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_5_3.midi
19788
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_6_1.midi
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_6_3.midi
19791
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_7_1.midi
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_7_3.midi
19794
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_8_1.midi
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_8_3.midi
19797
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_9_1.midi
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_9_3.midi
19800
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_10_1.midi
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_10_3.midi
19803
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_11_1.midi
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_11_3.midi
19806
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_12_1.midi
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_12_3.midi
19809
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_13_1.midi
5367f4a6a774be47a47b9b8c40a5605b_7064.midi_13_3.midi
19

20355
32de1e18b308d553fc39d4f43add6b4a_18494.midi_11_1.midi
32de1e18b308d553fc39d4f43add6b4a_18494.midi_11_3.midi
20358
32de1e18b308d553fc39d4f43add6b4a_18494.midi_12_1.midi
32de1e18b308d553fc39d4f43add6b4a_18494.midi_12_3.midi
20361
32de1e18b308d553fc39d4f43add6b4a_18494.midi_13_1.midi
32de1e18b308d553fc39d4f43add6b4a_18494.midi_13_3.midi
20364
32de1e18b308d553fc39d4f43add6b4a_18494.midi_14_1.midi
32de1e18b308d553fc39d4f43add6b4a_18494.midi_14_3.midi
20367
32de1e18b308d553fc39d4f43add6b4a_18494.midi_15_1.midi
32de1e18b308d553fc39d4f43add6b4a_18494.midi_15_3.midi
20370
32de1e18b308d553fc39d4f43add6b4a_18494.midi_16_1.midi
32de1e18b308d553fc39d4f43add6b4a_18494.midi_16_3.midi
20373
32de1e18b308d553fc39d4f43add6b4a_18494.midi_17_1.midi
32de1e18b308d553fc39d4f43add6b4a_18494.midi_17_3.midi
20376
32de1e18b308d553fc39d4f43add6b4a_18494.midi_18_1.midi
32de1e18b308d553fc39d4f43add6b4a_18494.midi_18_3.midi
20379
32de1e18b308d553fc39d4f43add6b4a_18494.midi_19_1.midi
32de1e18b308d553fc39d4f43add

20613
01aa9d4c699828e20dbc78f0fde71152_18842.midi_4_1.midi
01aa9d4c699828e20dbc78f0fde71152_18842.midi_4_3.midi
20616
01aa9d4c699828e20dbc78f0fde71152_18842.midi_5_1.midi
01aa9d4c699828e20dbc78f0fde71152_18842.midi_5_3.midi
20619
01aa9d4c699828e20dbc78f0fde71152_18842.midi_6_1.midi
01aa9d4c699828e20dbc78f0fde71152_18842.midi_6_3.midi


In [8]:
def evaluate_sim_and_final_create_dataset(pandas_dataframe, save_songs=True, return_similarity_array=False):
    dataframe = pandas_dataframe
    similarities = [0]*len(dataframe)
    count = 0
    similarities = np.empty([1,1000])
    for index,row in dataframe.iterrows():
        #print(index,row)
        if dataframe['type'][index]=='init':
            print(index)
            harmony_1 = dataframe['harmonic_reduction'][index]
            print(dataframe['midi_name'][index])
            harmony_2 = dataframe['harmonic_reduction'][index+2]
            print(dataframe['midi_name'][index+2])

            vec_1 = vectorize_harmony(model, harmony_1)
            vec_2 = vectorize_harmony(model, harmony_2)

            sim_score = cosine_similarity(vec_1, vec_2)
            #print(sim_score)
            #similarities[count]=sim_score
            count+=1
            if save_songs:
                if sim_score <= 0.2:
                    create_multitrack_1 = Multitrack(tracks=[Track(pianoroll=dataframe['pianoroll'][index])])
                    create_multitrack_2 = Multitrack(tracks=[Track(pianoroll=dataframe['pianoroll'][index+1])])
                    create_multitrack_3 = Multitrack(tracks=[Track(pianoroll=dataframe['pianoroll'][index+2])])

                    create_multitrack_1.write('after_sim_0.2/'+dataframe['midi_name'][index])
                    create_multitrack_2.write('after_sim_0.2/'+dataframe['midi_name'][index+1])
                    create_multitrack_3.write('after_sim_0.2/'+dataframe['midi_name'][index+2])
    #if return_similarity_array:
    #return similarities

In [9]:
evaluate_sim_and_final_create_dataset(test_df)

0
0bc945bdec9b3e295591dec504ebae13_12413.midi_0_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_0_3.midi
3
0bc945bdec9b3e295591dec504ebae13_12413.midi_1_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_1_3.midi
6
0bc945bdec9b3e295591dec504ebae13_12413.midi_2_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_2_3.midi
9
0bc945bdec9b3e295591dec504ebae13_12413.midi_3_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_3_3.midi
12
0bc945bdec9b3e295591dec504ebae13_12413.midi_4_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_4_3.midi
15
0bc945bdec9b3e295591dec504ebae13_12413.midi_5_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_5_3.midi
18
0bc945bdec9b3e295591dec504ebae13_12413.midi_6_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_6_3.midi
21
0bc945bdec9b3e295591dec504ebae13_12413.midi_7_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_7_3.midi
24
0bc945bdec9b3e295591dec504ebae13_12413.midi_8_1.midi
0bc945bdec9b3e295591dec504ebae13_12413.midi_8_3.midi
27
0bc945bdec9b3e295591

/home/pablo/miniconda3/envs/ai37/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


138
5386ffb3b559bebd66657123275688d7_453.midi_6_1.midi
5386ffb3b559bebd66657123275688d7_453.midi_6_3.midi
141
5386ffb3b559bebd66657123275688d7_453.midi_7_1.midi
5386ffb3b559bebd66657123275688d7_453.midi_7_3.midi
144
4393b768a7347752216f5f99d42aa0b8_7681.midi_0_1.midi
4393b768a7347752216f5f99d42aa0b8_7681.midi_0_3.midi
147
4393b768a7347752216f5f99d42aa0b8_7681.midi_1_1.midi
4393b768a7347752216f5f99d42aa0b8_7681.midi_1_3.midi
150
4393b768a7347752216f5f99d42aa0b8_7681.midi_2_1.midi
4393b768a7347752216f5f99d42aa0b8_7681.midi_2_3.midi
153
4393b768a7347752216f5f99d42aa0b8_7681.midi_3_1.midi
4393b768a7347752216f5f99d42aa0b8_7681.midi_3_3.midi
156
4393b768a7347752216f5f99d42aa0b8_7681.midi_4_1.midi
4393b768a7347752216f5f99d42aa0b8_7681.midi_4_3.midi
159
4393b768a7347752216f5f99d42aa0b8_7681.midi_5_1.midi
4393b768a7347752216f5f99d42aa0b8_7681.midi_5_3.midi
162
4393b768a7347752216f5f99d42aa0b8_7681.midi_6_1.midi
4393b768a7347752216f5f99d42aa0b8_7681.midi_6_3.midi
165
4393b768a7347752216f5f99d42a

408
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_4_1.midi
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_4_3.midi
411
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_5_1.midi
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_5_3.midi
414
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_6_1.midi
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_6_3.midi
417
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_7_1.midi
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_7_3.midi
420
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_8_1.midi
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_8_3.midi
423
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_9_1.midi
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_9_3.midi
426
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_10_1.midi
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_10_3.midi
429
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_11_1.midi
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_11_3.midi
432
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_12_1.midi
89f8a0384a6c23b0f3e762d12f57f3fb_19027.midi_12_3.midi
435


684
eb2ec506d16ead3cc7ca0d07286fdcb2_5493.midi_8_1.midi
eb2ec506d16ead3cc7ca0d07286fdcb2_5493.midi_8_3.midi
687
eb2ec506d16ead3cc7ca0d07286fdcb2_5493.midi_9_1.midi
eb2ec506d16ead3cc7ca0d07286fdcb2_5493.midi_9_3.midi
690
eb2ec506d16ead3cc7ca0d07286fdcb2_5493.midi_10_1.midi
eb2ec506d16ead3cc7ca0d07286fdcb2_5493.midi_10_3.midi
693
eb2ec506d16ead3cc7ca0d07286fdcb2_5493.midi_11_1.midi
eb2ec506d16ead3cc7ca0d07286fdcb2_5493.midi_11_3.midi
696
eb2ec506d16ead3cc7ca0d07286fdcb2_5493.midi_12_1.midi
eb2ec506d16ead3cc7ca0d07286fdcb2_5493.midi_12_3.midi
699
eb2ec506d16ead3cc7ca0d07286fdcb2_5493.midi_13_1.midi
eb2ec506d16ead3cc7ca0d07286fdcb2_5493.midi_13_3.midi
702
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_0_1.midi
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_0_3.midi
705
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_1_1.midi
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_1_3.midi
708
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_2_1.midi
97d3bb31e9f294cec561de85dd99e5cd_5155.midi_2_3.midi
711
97d3bb31e9f294ce

...

18474
f973ccfa78fb848452a4a1bf48df938f_5451.midi_3_1.midi
f973ccfa78fb848452a4a1bf48df938f_5451.midi_3_3.midi
18477
f973ccfa78fb848452a4a1bf48df938f_5451.midi_4_1.midi
f973ccfa78fb848452a4a1bf48df938f_5451.midi_4_3.midi
18480
f973ccfa78fb848452a4a1bf48df938f_5451.midi_5_1.midi
f973ccfa78fb848452a4a1bf48df938f_5451.midi_5_3.midi
18483
f973ccfa78fb848452a4a1bf48df938f_5451.midi_6_1.midi
f973ccfa78fb848452a4a1bf48df938f_5451.midi_6_3.midi
18486
f973ccfa78fb848452a4a1bf48df938f_5451.midi_7_1.midi
f973ccfa78fb848452a4a1bf48df938f_5451.midi_7_3.midi
18489
f973ccfa78fb848452a4a1bf48df938f_5451.midi_8_1.midi
f973ccfa78fb848452a4a1bf48df938f_5451.midi_8_3.midi
18492
f973ccfa78fb848452a4a1bf48df938f_5451.midi_9_1.midi
f973ccfa78fb848452a4a1bf48df938f_5451.midi_9_3.midi
18495
f973ccfa78fb848452a4a1bf48df938f_5451.midi_10_1.midi
f973ccfa78fb848452a4a1bf48df938f_5451.midi_10_3.midi
18498
8d82f8b359129dd9076c49f9c5eae874_8895.midi_0_1.midi
8d82f8b359129dd9076c49f9c5eae874_8895.midi_0_3.midi
18501
8d

19032
ef760a43e1a78139fe67aef58340210a_1833.midi_1_1.midi
ef760a43e1a78139fe67aef58340210a_1833.midi_1_3.midi
19035
ef760a43e1a78139fe67aef58340210a_1833.midi_2_1.midi
ef760a43e1a78139fe67aef58340210a_1833.midi_2_3.midi
19038
ef760a43e1a78139fe67aef58340210a_1833.midi_3_1.midi
ef760a43e1a78139fe67aef58340210a_1833.midi_3_3.midi
19041
ef760a43e1a78139fe67aef58340210a_1833.midi_4_1.midi
ef760a43e1a78139fe67aef58340210a_1833.midi_4_3.midi
19044
ef760a43e1a78139fe67aef58340210a_1833.midi_5_1.midi
ef760a43e1a78139fe67aef58340210a_1833.midi_5_3.midi
19047
ef760a43e1a78139fe67aef58340210a_1833.midi_6_1.midi
ef760a43e1a78139fe67aef58340210a_1833.midi_6_3.midi
19050
ae2f62389dcf88905f676e8f9e85c22e_7740.midi_0_1.midi
ae2f62389dcf88905f676e8f9e85c22e_7740.midi_0_3.midi
19053
ae2f62389dcf88905f676e8f9e85c22e_7740.midi_1_1.midi
ae2f62389dcf88905f676e8f9e85c22e_7740.midi_1_3.midi
19056
ae2f62389dcf88905f676e8f9e85c22e_7740.midi_2_1.midi
ae2f62389dcf88905f676e8f9e85c22e_7740.midi_2_3.midi
19059
ae2f

19485
f3afc03060463c0d91118e99a858dd94_13570.midi_2_1.midi
f3afc03060463c0d91118e99a858dd94_13570.midi_2_3.midi
19488
f3afc03060463c0d91118e99a858dd94_13570.midi_3_1.midi
f3afc03060463c0d91118e99a858dd94_13570.midi_3_3.midi
19491
f3afc03060463c0d91118e99a858dd94_13570.midi_4_1.midi
f3afc03060463c0d91118e99a858dd94_13570.midi_4_3.midi
19494
f3afc03060463c0d91118e99a858dd94_13570.midi_5_1.midi
f3afc03060463c0d91118e99a858dd94_13570.midi_5_3.midi
19497
f3afc03060463c0d91118e99a858dd94_13570.midi_6_1.midi
f3afc03060463c0d91118e99a858dd94_13570.midi_6_3.midi
19500
f3afc03060463c0d91118e99a858dd94_13570.midi_7_1.midi
f3afc03060463c0d91118e99a858dd94_13570.midi_7_3.midi
19503
f3afc03060463c0d91118e99a858dd94_13570.midi_8_1.midi
f3afc03060463c0d91118e99a858dd94_13570.midi_8_3.midi
19506
f3afc03060463c0d91118e99a858dd94_13570.midi_9_1.midi
f3afc03060463c0d91118e99a858dd94_13570.midi_9_3.midi
19509
f3afc03060463c0d91118e99a858dd94_13570.midi_10_1.midi
f3afc03060463c0d91118e99a858dd94_13570.midi_

19947
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_16_1.midi
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_16_3.midi
19950
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_17_1.midi
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_17_3.midi
19953
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_18_1.midi
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_18_3.midi
19956
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_19_1.midi
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_19_3.midi
19959
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_20_1.midi
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_20_3.midi
19962
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_21_1.midi
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_21_3.midi
19965
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_22_1.midi
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_22_3.midi
19968
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_23_1.midi
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_23_3.midi
19971
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_24_1.midi
33f1e2818356a3780bfb8eb98cc4a7bd_6199.midi_24

20388
32de1e18b308d553fc39d4f43add6b4a_18494.midi_22_1.midi
32de1e18b308d553fc39d4f43add6b4a_18494.midi_22_3.midi
20391
32de1e18b308d553fc39d4f43add6b4a_18494.midi_23_1.midi
32de1e18b308d553fc39d4f43add6b4a_18494.midi_23_3.midi
20394
32de1e18b308d553fc39d4f43add6b4a_18494.midi_24_1.midi
32de1e18b308d553fc39d4f43add6b4a_18494.midi_24_3.midi
20397
32de1e18b308d553fc39d4f43add6b4a_18494.midi_25_1.midi
32de1e18b308d553fc39d4f43add6b4a_18494.midi_25_3.midi
20400
5c224dbb4dffa02476c6fe3ac9d7877a_11092.midi_0_1.midi
5c224dbb4dffa02476c6fe3ac9d7877a_11092.midi_0_3.midi
20403
5c224dbb4dffa02476c6fe3ac9d7877a_11092.midi_1_1.midi
5c224dbb4dffa02476c6fe3ac9d7877a_11092.midi_1_3.midi
20406
5c224dbb4dffa02476c6fe3ac9d7877a_11092.midi_2_1.midi
5c224dbb4dffa02476c6fe3ac9d7877a_11092.midi_2_3.midi
20409
5c224dbb4dffa02476c6fe3ac9d7877a_11092.midi_3_1.midi
5c224dbb4dffa02476c6fe3ac9d7877a_11092.midi_3_3.midi
20412
5c224dbb4dffa02476c6fe3ac9d7877a_11092.midi_4_1.midi
5c224dbb4dffa02476c6fe3ac9d7877a_1109

In [245]:
def get_related_chords(token, topn=3):
    print("Similar chords with " + token)
    for word, similarity in model.wv.most_similar(positive=[token], topn=topn):
        print (word, round(similarity, 3))

def get_chord_similarity(chordA, chordB):
    print("Similarity between {0} and {1}: {2}".format(
        chordA, chordB, model.wv.similarity(chordA, chordB)))
    
print("List of chords found:")
print(model.wv.vocab.keys())
print("Number of chords considered by model: {0}".format(len(model.wv.vocab)))

get_related_chords('i6')
get_related_chords('-VII')
#get_related_chords('vi65')
get_related_chords('i64')
get_related_chords('i53')
get_related_chords('III53')
#get_related_chords('v')

# The first one should be smaller since "i" and "ii" chord doesn't share notes,
# different from "IV" and "vi" which share 2 notes.
get_chord_similarity("V", "-V")
get_chord_similarity("V", "v") 

List of chords found:
dict_keys(['V53', 'IV53', 'v', 'I53', 'II7', 'V7', 'iii', 'v42', '-VII53', 'i', 'I', 'iv42', 'IV7', 'ii65', 'I64', 'vi42', 'IV64', 'I6', 'IV', 'v7', 'iv', 'V6', 'IV6', 'iii7', 'vi', 'ii7', 'iv43', 'V', 'vii6', 'V43', 'iii53', 'I7', 'III6', 'III65', 'VI64', 'v53', 'III53', 'II6', 'VI53', 'III7', '#vi', 'vii', '-VI53', 'v64', 'III', '-VII64', 'ii', 'v65', '--VII', '-VI', '#iii7', 'iii64', '#iii', '-i64', '-iv', '--iii', '--iii53', '--III53', '#vii65', '--vii', 'i53', '-i', '#iii53', 'iii65', '#iii65', 'i7', 'V64', 'I65', 'II65', '-VII', 'v43', 'III64', 'ii53', 'IV65', '-II65', '-v', '-v53', '-VI7', 'vi53', 'i64', 'iv65', 'iii42', 'I42', 'vii53', '-III65', 'i43', 'iii6', 'ii42', 'III43', 'v6', 'i6', 'iv53', '-VI65', '-VI6', 'vi6', '-VII6', '-V', 'i65', '-VI43', '#vi42', 'III42', 'ii43', '#vi7', 'vi43', 'VII', 'vi64', 'iv64', '#v53', '#V', 'ii64', '-ii53', '#ii42', 'IV42', '#iv42', 'vii42', 'I43', 'vii65', 'V65', 'vi65', '#iv6', 'VI', 'II53', '-VI64', '#vi6', 'II', 'I

In [35]:
# SAVE CSV
test_df.to_csv('dataset_interpolation_10', index=False)

In [76]:
jejeje = pd.read_csv('dataset_interpolation_10')

In [201]:
jejeje.head()

,midi_name,type,harmonic_reduction,pianoroll
0,0bc945bdec9b3e295591dec504ebae13_12413.midi_0_...,init,"[V53, IV53, V53, IV53, V53, v]",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...
1,0bc945bdec9b3e295591dec504ebae13_12413.midi_0_...,interpol,"[IV53, V53, I53, I53, IV53, II7]",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...
2,0bc945bdec9b3e295591dec504ebae13_12413.midi_0_...,end,"[V7, iii, v42, -VII53, i, IV53]",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...
3,0bc945bdec9b3e295591dec504ebae13_12413.midi_1_...,init,"[IV53, V53, I53, I53, IV53, II7]",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...
4,0bc945bdec9b3e295591dec504ebae13_12413.midi_1_...,interpol,"[V7, iii, v42, -VII53, i, IV53]",[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...


In [204]:
(jejeje['harmonic_reduction'][0])

['V53', 'IV53', 'V53', 'IV53', 'V53', 'v']

In [160]:
dic = {'pianoroll': []} #create empty dictionary

song_name = os.listdir('lahk_midi_tracks')[0]
print(song_name)
mid_multitrack = Multitrack(filename='lahk_midi_tracks/'+song_name)
print(mid_multitrack)
merged_pianoroll_total = mid_multitrack.get_merged_pianoroll(mode='sum') # merge all tracks #sum or max
roll_1 = merged_pianoroll_total[0:500] #partial segments of pianoroll
dic['pianoroll'].append(roll_1)

0bc945bdec9b3e295591dec504ebae13_12413.midi
tracks : [Piano],
tempo : [120. 120. 120. ... 120. 120. 120.],
downbeat : [ True False False ... False False False],
beat_resolution : 24,
name : unknown


In [177]:
d = pd.DataFrame(data=dic)
d['pianoroll'][0].shape #500,128
#type(d['pianoroll'][0])

(d['pianoroll'][0][0])

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0, 71,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 61,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=uint64)

In [191]:
case = (jejeje['pianoroll'][0])
case = case.replace('"', '')
print(case)
#jejeje['pianoroll'].transform(ast.literal_eval)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [171]:
import re
import ast

In [135]:
# solve string format problem in harmonic_reduction: string to list of strings
case = "['V53', 'IV53', 'V53', 'IV53', 'V53', 'v']"
case = case.replace('"', '')
case = case.replace("'", '')
res = case.strip('][').split(', ')

In [136]:
maxi = len(jejeje)
for i in range(0,maxi):
    case = jejeje['harmonic_reduction'][i]
    case = case.replace('"', '')
    case = case.replace("'", '')
    jejeje['harmonic_reduction'][i] = case.strip('][').split(', ')

In [217]:
for i in range(0,len(jejeje)):
    #print(jejeje['midi_name'][i])
    if jejeje['midi_name'][i] == '0c2600766d5c277b6e66692e4007d0d5_1133.midi_15_1.midi':
        print(jejeje['midi_name'][i])
        print(jejeje['harmonic_reduction'][i])
        print(jejeje['midi_name'][i+2])
        print(jejeje['harmonic_reduction'][i+2])
    if jejeje['midi_name'][i] == '01aa9d4c699828e20dbc78f0fde71152_18842.midi_5_1.midi':
        print(jejeje['midi_name'][i])
        print(jejeje['harmonic_reduction'][i])
        print(jejeje['midi_name'][i+2])
        print(jejeje['harmonic_reduction'][i+2])

0c2600766d5c277b6e66692e4007d0d5_1133.midi_15_1.midi
['i6', '-VII', 'i64', 'i53', 'III53', 'III53']
0c2600766d5c277b6e66692e4007d0d5_1133.midi_15_3.midi
['vi64', 'vi6', 'I', 'v', 'V53', '-v']
01aa9d4c699828e20dbc78f0fde71152_18842.midi_5_1.midi
['vi65', 'V6', '#i42', 'iii65', 'iii7', 'vi7']
01aa9d4c699828e20dbc78f0fde71152_18842.midi_5_3.midi
['III64', 'III', 'i42', 'i', 'ii42', 'i64']
